<a href="https://colab.research.google.com/github/Kadakol/ML/blob/master/NYU%20Deep%20Learning%20DS-GA%201008/Spring%202020/Practicum/09_echo_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import numpy as np

class EchoData():

    def __init__(self, series_length=40000, batch_size=32,
                 echo_step=3, truncated_length=10, seed=None):

        self.series_length = series_length
        self.truncated_length = truncated_length
        self.n_batches = series_length//truncated_length

        self.echo_step = echo_step
        self.batch_size = batch_size
        if seed is not None:
            np.random.seed(seed)
        self.x_batch = None
        self.y_batch = None
        self.x_chunks = []
        self.y_chunks = []
        self.generate_new_series()
        self.prepare_batches()

    def __getitem__(self, index):
        if index == 0:
            self.generate_new_series()
            self.prepare_batches()
        return self.x_chunks[index], self.y_chunks[index]

    def __len__(self):
        return self.n_batches

    def generate_new_series(self):
        x = np.random.choice(
            2,
            size=(self.batch_size, self.series_length),
            p=[0.5, 0.5])
        y = np.roll(x, self.echo_step, axis=1)
        y[:, 0:self.echo_step] = 0
        self.x_batch = x
        self.y_batch = y

    def prepare_batches(self):
        x = np.expand_dims(self.x_batch, axis=-1)
        y = np.expand_dims(self.y_batch, axis=-1)
        self.x_chunks = np.split(x, self.n_batches, axis=1)
        self.y_chunks = np.split(y, self.n_batches, axis=1)

In [2]:
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1);

In [3]:
batch_size = 5
echo_step = 3
series_length = 20_000
BPTT_T = 20

train_data = EchoData(
    echo_step=echo_step,
    batch_size=batch_size,
    series_length=series_length,
    truncated_length=BPTT_T,
)
train_size = len(train_data)

test_data = EchoData(
    echo_step=echo_step,
    batch_size=batch_size,
    series_length=series_length,
    truncated_length=BPTT_T,
)
test_size = len(test_data)

In [4]:
# Let's print first 20 timesteps of the first sequences to see the echo data:
print('(1st input sequence)  x:', *train_data.x_batch[0, :20], '... ')
print('(1st target sequence) y:', *train_data.y_batch[0, :20], '... ')

(1st input sequence)  x: 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 0 ... 
(1st target sequence) y: 0 0 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 ... 


In [5]:
# batch_size different sequences are created:
print('x_batch:', *(str(d)[1:-1] + ' ...' for d in train_data.x_batch[:, :20]), sep='\n')
print('x_batch size:', train_data.x_batch.shape)
print()
print('y_batch:', *(str(d)[1:-1] + ' ...' for d in train_data.y_batch[:, :20]), sep='\n')
print('y_batch size:', train_data.y_batch.shape)

x_batch:
1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 0 ...
0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 ...
1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1 ...
1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1 ...
0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 0 ...
x_batch size: (5, 20000)

y_batch:
0 0 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 ...
0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 ...
0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 ...
0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 ...
0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 ...
y_batch size: (5, 20000)


In [6]:
# In order to use RNNs data is organized into temporal
# chunks of size [batch_size, T, feature_dim]
print('x_chunk:', *train_data.x_chunks[0].squeeze(), sep='\n')
print('1st x_chunk size:', train_data.x_chunks[0].shape)
print()
print('y_chunk:', *train_data.y_chunks[0].squeeze(), sep='\n')
print('1st y_chunk size:', train_data.y_chunks[0].shape)

x_chunk:
[1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
[1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1]
[1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1]
[0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 0]
1st x_chunk size: (5, 20, 1)

y_chunk:
[0 0 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1]
[0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1]
[0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0]
[0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1]
1st y_chunk size: (5, 20, 1)


In [7]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, rnn_hidden_size, output_size):
        super().__init__()
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = torch.nn.RNN(
            input_size=input_size,
            hidden_size=rnn_hidden_size,
            num_layers=1,
            nonlinearity='relu',
            batch_first=True
        )
        self.linear = torch.nn.Linear(
            in_features=rnn_hidden_size,
            out_features=1
        )

    def forward(self, x, hidden):
        x, hidden = self.rnn(x, hidden)  
        x = self.linear(x)
        return x, hidden

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
def train(hidden):
    model.train()
       
    correct = 0
    for batch_idx in range(train_size):
        data, target = train_data[batch_idx]
        data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
        optimizer.zero_grad()
        if hidden is not None: hidden.detach_()
        logits, hidden = model(data, hidden)
        loss = criterion(logits, target)
        loss.backward()
        optimizer.step()
        
        pred = (torch.sigmoid(logits) > 0.5)
        correct += (pred == target.byte()).int().sum().item()
        
    return correct, loss.item(), hidden

In [10]:
def test(hidden):
    model.eval()   
    correct = 0
    with torch.no_grad():
        for batch_idx in range(test_size):
            data, target = test_data[batch_idx]
            data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
            logits, hidden = model(data, hidden)
            
            pred = (torch.sigmoid(logits) > 0.5)
            correct += (pred == target.byte()).int().sum().item()

    return correct

In [11]:
feature_dim = 1 #since we have a scalar series
h_units = 4

model = SimpleRNN(
    input_size=1,
    rnn_hidden_size=h_units,
    output_size=feature_dim
).to(device)
hidden = None
        
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [12]:
n_epochs = 5
epoch = 0

while epoch < n_epochs:
    correct, loss, hidden = train(hidden)
    epoch += 1
    train_accuracy = float(correct) / train_size
    print(f'Train Epoch: {epoch}/{n_epochs}, loss: {loss:.3f}, accuracy {train_accuracy:.1f}%')

#test    
correct = test(hidden)
test_accuracy = float(correct) / test_size
print(f'Test accuracy: {test_accuracy:.1f}%')

Train Epoch: 1/5, loss: 0.707, accuracy 57.1%
Train Epoch: 2/5, loss: 0.557, accuracy 65.2%
Train Epoch: 3/5, loss: 0.435, accuracy 73.0%
Train Epoch: 4/5, loss: 0.325, accuracy 80.6%
Train Epoch: 5/5, loss: 0.344, accuracy 83.7%
Test accuracy: 84.2%


In [13]:
# Let's try some echoing
my_input = torch.empty(1, 100, 1).random_(2).to(device)
hidden = None
my_out, _ = model(my_input, hidden)
print(my_input.view(1, -1).byte(), (my_out > 0).view(1, -1), sep='\n')

tensor([[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
         0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
         0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
         1, 1, 0, 0]], dtype=torch.uint8)
tensor([[ True, False, False,  True, False, False,  True,  True, False, False,
          True, False, False, False,  True,  True, False,  True,  True, False,
         False,  True, False, False,  True,  True,  True, False,  True, False,
         False,  True,  True, False,  True, False, False,  True, False, False,
          True,  True, False,  True,  True, False,  True,  True,  True, False,
          True, False, False,  True,  True, False,  True,  True, False,  True,
         False, False,  True,  True,  True, False,  True,  True, False,  True,
          True, False,  True, False,  True,  True, False, False,  True, False,
  